## File sorting

The cells below will move data-cube stats tile based results into folders labelled by year. Need to first manually sort the results into discretely labelled folders based on the dcstats query. ie move all the ndvi_max results to a folder called 'ndvi_max', all the ndvi_pct files to a folder called 'ndvi_pct' etc. 

In [4]:
import fnmatch
import os
from multiprocessing import Pool

In [1]:
#create a list of years in format "19901101.tif" so 
#we can aggregate files by year
z = range(2018,2019, 1)
years = []
for zz in z:
    years.append('*' + str(zz) + '1101.tif')

In [2]:
years

['*20181101.tif']

In [ ]:
directory='results/mdb_NSW/summer1989/ndvi_max/'
for file in os.listdir(directory):
    for year in years:
        if not os.path.exists(directory + str(year[1:5])):
            os.mkdir(directory +str(year[1:5])) #create folders of the years
        if fnmatch.fnmatch(file, year): #find matching years and move them to the correct folder
            os.rename(directory +file, directory + year[1:5] + '/' + file)

In [5]:
directory='results/mdb_NSW/summer2019/ndviArgMaxMin/'
for file in os.listdir(directory):
    for year in years:
        if not os.path.exists(directory + str(year[1:5])):
            os.mkdir(directory +str(year[1:5]))
        if fnmatch.fnmatch(file, year): 
            os.rename(directory +file, directory + year[1:5] + '/' + file)

In [ ]:
directory='results/mdb_NSW/summer/ndvi_pct/'
for file in os.listdir(directory):
    for year in years:
        if not os.path.exists(directory + str(year[1:5])):
            os.mkdir(directory +str(year[1:5]))
        if fnmatch.fnmatch(file, year): 
            os.rename(directory +file, directory + year[1:5] + '/' + file)

#### to reverse the above operations

In [ ]:
!find src/dir/ -name '*.tif' -exec mv {} target/dir/ \;

#### changing names of mosaic file

In [ ]:
z = range(1990,2018, 1)
years = []
for zz in z:
    years.append('ndvi_max' + str(zz) + '1101_mosaic.tif')

directory='results/mdb_NSW/summer/ndviArgMaxMin/mosaics/'
files = os.listdir(directory)
files.sort()

for file in files:
    for year in years:
        try:
            os.rename(directory + file+'/'+ year, directory + file + '/' + 'ndvi_max_' + year[0:4]+'1101_mosaic.tif')
        except:
            pass

## Build .vrt and mosaic tiff for each year

The following couple of cells will create .vrt files (which are used to read large numbers of geotiffs into QGIS), then, using the .vrt file,
gdal is used to create a sigle mosaicked geotiff of all the tiles for each year.  Beware: this can be very slow if you have compressed multi-band
geotiffs. An executable python file that takes advantage of multithreading is available for large jobs (build_vrt_and_mosaic.py).

In [7]:
#build vrt
directory = "/g/data/r78/cb3058/dea-notebooks/dcStats/results/mdb_NSW/summer87_90/ndviArgMaxMin/"
for folder in os.listdir(directory):
    os.chdir(directory + folder)
    os.system("gdalbuildvrt " + "ndviArgMaxMin_" + folder + "1101_mosaic.vrt *.tif")

In [ ]:
#build mosaic
for folder in os.listdir(directory):
    os.chdir(directory + folder)
    os.system("gdal_translate "\
       "-co BIGTIFF=YES "\
       "-co COMPRESS=DEFLATE "\
       "-co ZLEVEL=9 "\
       "-co PREDICTOR=1 "\
       "-co TILED=YES "\
       "-co BLOCKXSIZE=1024 "\
       "-co BLOCKYSIZE=1024 "\
       + "ndvi_max_" + folder + "1101_mosaic.vrt " + "ndvi_max_" + folder + "1101_mosaic.tif")

In [9]:
#multithreaded version of mosaic
cpus=2
directory = "/g/data/r78/cb3058/dea-notebooks/dcStats/results/mdb_NSW/summer87_90/ndvi_max/"
def mosaicTiffs(folder):
    os.chdir(directory + folder)
    os.system("gdal_translate "\
       "-co BIGTIFF=YES "\
       "-co COMPRESS=DEFLATE "\
       "-co ZLEVEL=9 "\
       "-co PREDICTOR=1 "\
       "-co TILED=YES "\
       "-co BLOCKXSIZE=1024 "\
       "-co BLOCKYSIZE=1024 "\
       + "ndvi_max_" + folder + "1101_mosaic.vrt " + "ndvi_max_" + folder + "1101_mosaic.tif")

folder = os.listdir(directory)

if __name__ == '__main__':
    pool = Pool(cpus)  
    pool.map(mosaicTiffs, folder)